In [ ]:
import re
import requests
from bs4 import BeautifulSoup
import json
import scipy
import numpy as np
import pandas as pd
import sklearn
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
url = 'http://bj.bendibao.com/ditie/linemap.shtml'
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) ' \
             'Chrome/77.0.3865.90 Safari/537.36'

In [3]:
#抓取网页html
def get_html(url, user_agent):
    s = requests.Session()
    s.headers['User-Agent'] = user_agent
    html = s.get(url)
    html.encoding = html.apparent_encoding
    soup = BeautifulSoup(html.text, 'html.parser')
    return soup

In [4]:
get_html(url, user_agent)


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>北京地铁线路图，2019最新北京地铁线路图，北京地铁地图-北京本地宝</title>
<meta content="北京地铁线路图,2019北京地铁最新线路图,北京地铁地图" name="Keywords"/>
<meta content="北京本地宝交通频道为你带来北京地铁线路图，2019北京地铁最新线路图的详细信息" name="Description"/>
<meta content="webkit" name="renderer"/>
<link href="http://jtapi.bendibao.com/ditie/css/cssreset.min.css" rel="stylesheet"/>
<link href="http://jtapi.bendibao.com/ditie/css/metro.min.css" rel="stylesheet"/>
<link href="http://jtapi.bendibao.com/ditie/css/linecolor.min.css" rel="stylesheet"/>
<link href="http://jtapi.bendibao.com/ditie/css/timetable.min.css" rel="stylesheet"/>
<link href="http://jtapi.bendibao.com/ditie/css/top.css" rel="stylesheet"/>
<meta content="format=xhtml; url=http://m.bj.bendibao.com/ditie/linemap.shtml" http-equiv="mobile-agent">
<met

In [6]:
#抓取处理北京各条地铁线路数据
# 返回一个字典，{line_name: [station1，station2，...]}
def get_line(url,user_agent):
 
    soup = get_html(url, user_agent)
    subway = {}
    line_1= soup.find_all('div', class_='wrap')

    for content in line_1:
        line = content.text.split()[0]
        links = 'http://bj.bendibao.com'+(str(content.find('a').attrs.get("href")))
        subway[line]=links

    reg = r'◆◆(.*)'
    sub_line = {}
    reg_l =  r'北京地铁(.*)线路图'

    for sp in subway.keys():
        l = re.findall(reg_l, sp)[0]
        if '2号线' in l:
            l = '2号线'
        url_1 = subway[sp]
        soup = get_html(url_1,user_agent)
        sub_line[l] = []

        stops = soup.find_all('div', class_='station')
        for s in stops:
            if '你可以在此处换乘 ：' in s.text:
                sub_line[l].append(re.findall(reg, s.text)[0])
            else:
                sub_line[l].append(s.text)

    return sub_line


In [10]:
sub_line=get_line(url,user_agent)

In [13]:
sub_line

{'1号线(M1)': ['苹果园',
  '古城',
  '八角游乐园',
  '八宝山',
  '玉泉路',
  '五棵松',
  '万寿路',
  '公主坟',
  '军事博物馆',
  '木樨地',
  '南礼士路',
  '复兴门',
  '西单',
  '天安门西',
  '天安门东',
  '王府井',
  '东单',
  '建国门',
  '永安里',
  '国贸',
  '大望路',
  '四惠',
  '四惠东'],
 '2号线': ['西直门',
  '车公庄',
  '阜成门',
  '复兴门',
  '长椿街',
  '宣武门',
  '和平门',
  '前门',
  '崇文门',
  '北京站',
  '建国门',
  '朝阳门',
  '东四十条',
  '东直门',
  '雍和宫',
  '安定门',
  '鼓楼大街',
  '积水潭'],
 '4号线(M4)': ['安河桥北',
  '北宫门',
  '西苑',
  '圆明园',
  '北京大学东门',
  '中关村',
  '海淀黄庄',
  '人民大学',
  '魏公村',
  '国家图书馆',
  '动物园',
  '西直门',
  '新街口',
  '平安里',
  '西四',
  '灵境胡同',
  '西单',
  '宣武门',
  '菜市口',
  '陶然亭',
  '北京南站',
  '马家堡',
  '角门西',
  '公益西桥',
  '新宫',
  '西红门',
  '高米店北',
  '高米店南',
  '枣园',
  '清源路',
  '黄村西大街',
  '黄村火车站',
  '义和庄',
  '生物医药基地',
  '天宫院'],
 '5号线(M5)': ['宋家庄',
  '刘家窑',
  '蒲黄榆',
  '天坛东门',
  '磁器口',
  '崇文门',
  '东单',
  '灯市口',
  '东四',
  '张自忠路',
  '北新桥',
  '雍和宫',
  '和平里北街',
  '和平西桥',
  '惠新西街南口',
  '惠新西街北口',
  '大屯路东',
  '北苑路北',
  '立水桥南',
  '立水桥',
  '天通苑南',
  '天通苑',
  '天通苑北'],
 '6号线': ['海淀五路居',
  '慈寿寺',
  '花园桥

In [8]:
#生成地铁站，地铁线路和换乘数据集
def build_station_network(sub_line, user_agent):
    '''
    :param sub_line: input_dict {line_name: [station1，station2，...]}
    :param user_agent: user agent
    :return: output_dict {station:{neighbor1:line_name,neighbor2:line_name,...},station2:{...},...}
    '''
    net = {}
    for key in (sub_line.keys()):
        for i in range(len(sub_line[key])):
            if sub_line[key][i] not in net:
                net[sub_line[key][i]] = {}
            if i > 0 and i < len(sub_line[key]) - 1:
                net[sub_line[key][i]][sub_line[key][i - 1]] = key
                net[sub_line[key][i]][sub_line[key][i + 1]] = key
            elif i == 0:
                net[sub_line[key][i]][sub_line[key][i + 1]] = key
            elif i == len(sub_line[key]) - 1:
                net[sub_line[key][i]][sub_line[key][i - 1]] = key

    #update line2
    net['西直门']['积水潭'] = '2号线'
    net['积水潭']['西直门'] = '2号线'

    return net

In [14]:
net=build_station_network(sub_line, user_agent)

In [15]:
net

{'苹果园': {'古城': '1号线(M1)'},
 '古城': {'苹果园': '1号线(M1)', '八角游乐园': '1号线(M1)'},
 '八角游乐园': {'古城': '1号线(M1)', '八宝山': '1号线(M1)'},
 '八宝山': {'八角游乐园': '1号线(M1)', '玉泉路': '1号线(M1)'},
 '玉泉路': {'八宝山': '1号线(M1)', '五棵松': '1号线(M1)'},
 '五棵松': {'玉泉路': '1号线(M1)', '万寿路': '1号线(M1)'},
 '万寿路': {'五棵松': '1号线(M1)', '公主坟': '1号线(M1)'},
 '公主坟': {'万寿路': '1号线(M1)', '军事博物馆': '1号线(M1)', '莲花桥': '10号线', '西钓鱼台': '10号线'},
 '军事博物馆': {'公主坟': '1号线(M1)',
  '木樨地': '1号线(M1)',
  '北京西站': '9号线(M9)',
  '白堆子': '9号线(M9)'},
 '木樨地': {'军事博物馆': '1号线(M1)', '南礼士路': '1号线(M1)'},
 '南礼士路': {'木樨地': '1号线(M1)', '复兴门': '1号线(M1)'},
 '复兴门': {'南礼士路': '1号线(M1)', '西单': '1号线(M1)', '阜成门': '2号线', '长椿街': '2号线'},
 '西单': {'复兴门': '1号线(M1)',
  '天安门西': '1号线(M1)',
  '灵境胡同': '4号线(M4)',
  '宣武门': '4号线(M4)'},
 '天安门西': {'西单': '1号线(M1)', '天安门东': '1号线(M1)'},
 '天安门东': {'天安门西': '1号线(M1)', '王府井': '1号线(M1)'},
 '王府井': {'天安门东': '1号线(M1)', '东单': '1号线(M1)'},
 '东单': {'王府井': '1号线(M1)',
  '建国门': '1号线(M1)',
  '崇文门': '5号线(M5)',
  '灯市口': '5号线(M5)'},
 '建国门': {'东单': '1号线(M1)', '永安里': '1号

In [16]:
net['奥林匹克公园']

{'森林公园南门': '8号线(M8)', '奥体中心': '8号线(M8)', '安立路': '15号线', '北沙滩': '15号线'}

In [30]:
# 初始bfs算法
def bfs_class(graph,start,destination):
    pathes = [[start]]
    visited=set()
    
    while pathes:
        path=pathes.pop(0)
        froniter=path[-1]
        
        if froniter in visited:continue
            
        successors=graph[froniter]
        for station,line in successors.items():
            if station in path:continue
            new_path=path+[station]
            pathes.append(new_path)
            if(station==destination):
                return new_path
        visited.add(froniter)

In [31]:
bfs_class(net, '北沙滩', '东直门')

['北沙滩', '奥林匹克公园', '奥体中心', '北土城', '安华桥', '鼓楼大街', '安定门', '雍和宫', '东直门']

In [53]:
# 初始dfs算法
def dfs_class(graph,start,destination):
    pathes = [[start]]
    visited=set()
    
    while pathes:
        path=pathes.pop(0)
        froniter=path[-1]
        if froniter in visited:continue
            
        successors=graph[froniter]
        for station,line in successors.items():
            if station in path:continue
            new_path=path+[station]
            pathes=[new_path]+pathes
            if(station==destination):
                return new_path
        visited.add(froniter)

In [95]:
dfs_class(net, '北沙滩', '东直门')

['北沙滩',
 '奥林匹克公园',
 '安立路',
 '关庄',
 '望京西',
 '望京',
 '阜通',
 '望京南',
 '将台',
 '东风北桥',
 '枣营',
 '金台路',
 '大望路',
 '九龙山',
 '北工大西门',
 '十里河',
 '方庄',
 '蒲黄榆',
 '景泰',
 '永定门外',
 '北京南站',
 '马家堡',
 '角门西',
 '草桥',
 '纪家庙',
 '首经贸',
 '丰台站',
 '泥洼',
 '西局',
 '七里庄',
 '六里桥',
 '莲花桥',
 '公主坟',
 '西钓鱼台',
 '慈寿寺',
 '花园桥',
 '白石桥南',
 '国家图书馆',
 '动物园',
 '西直门',
 '积水潭',
 '鼓楼大街',
 '什刹海',
 '南锣鼓巷',
 '东四',
 '朝阳门',
 '东大桥',
 '呼家楼',
 '团结湖',
 '农业展览馆',
 '亮马桥',
 '三元桥',
 '东直门']

In [82]:
# BFS with search strategy(alterable step cost)
def standard_bfs_0(graph,start,destination,search_strategy):
    visited=set()
    transfer=set()
    pathes=[[[start],transfer]]
    while pathes:
        path_wtrans=pathes.pop(0)
        path=path_wtrans[0]
        transfer=path_wtrans[1]
        frontier=path[-1]
        if frontier in visited:continue
        successors=graph[frontier]
        for station in successors.items():
            if station[0] in path: continue
            new_trans=transfer|{station[1]}
           # print("new_trans",new_trans)
            new_path=path+[station[0]]
            if station[0]==destination:
                #print("pathe1",pathes[0][0][-1])
                #print("pathe2",pathes[0][-1])
                #print("pathe2",pathes[0][0])
                #print("pathes",pathes)
                return 'path found with {} stations: {} with number of transfers: {}'.format(len(new_path)-1,new_path,len(new_trans)-1)
            pathes.append([new_path,new_trans]) 
        pathes=search_strategy(pathes)
        visited.add(frontier)

In [88]:
# BFS with search strategy(alterable step cost)
def standard_bfs(graph,start,destination,search_strategy):
    visited=set()
    transfer=set()
    pathes=[[[start],transfer]]
    while pathes:
        path_wtrans=pathes.pop(0)
        path=path_wtrans[0]
        transfer=path_wtrans[1]
        frontier=path[-1]
        if frontier in visited:continue
        if frontier == destination:
            #return path
            return 'path found with {} stations: {} with number of transfers: {}'.format(len(path)-1,path,len(path_wtrans[1])-1)
        successors=graph[frontier]
        for station in successors.items():
            if station[0] in path: continue
            new_trans=transfer|{station[1]}
            new_path=path+[station[0]]
            pathes.append([new_path,new_trans]) 
            
        pathes=search_strategy(pathes)
        visited.add(frontier)
        if pathes and (destination == pathes[0][0][-1]):
            return 'path found with {} stations: {} with number of transfers: {}'.format(len(pathes[0][0])-1,pathes[0][0],len(pathes[0][-1])-1)

In [89]:
%%time
standard_bfs(net, '北沙滩', '东直门',search_strategy=lambda x:x)

Wall time: 0 ns


"path found with 8 stations: ['北沙滩', '奥林匹克公园', '奥体中心', '北土城', '安华桥', '鼓楼大街', '安定门', '雍和宫', '东直门'] with number of transfers: 2"

In [83]:
%%time
standard_bfs_0(net, '北沙滩', '东直门',search_strategy=lambda x:x)

Wall time: 0 ns


"path found with 8 stations: ['北沙滩', '奥林匹克公园', '奥体中心', '北土城', '安华桥', '鼓楼大街', '安定门', '雍和宫', '东直门'] with number of transfers: 2"

####  最少换乘路线：假设换乘对结果影响非常大

In [91]:
def sort_by_combined_cost(pathes,transfer_cost=1000):  
    def combined_cost(path):
        return (len(path[0])-1)+(len(path[1])-1)*transfer_cost
    return sorted(pathes,key=combined_cost)
standard_bfs(net, '北沙滩', '东直门',search_strategy=sort_by_combined_cost)

"path found with 8 stations: ['北沙滩', '奥林匹克公园', '安立路', '关庄', '望京西', '芍药居', '光熙门', '柳芳', '东直门'] with number of transfers: 1"

####  最短距离路线：假设换乘对距离无影响

In [94]:
def sort_by_combined_cost(pathes,transfer_cost=0):  
    def combined_cost(path):
       # print("path[0]",path[0])
       # print("path[1]",path[1])
        return (len(path[0])-1)+(len(path[1])-1)*transfer_cost
    return sorted(pathes,key=combined_cost)
standard_bfs(net, '北沙滩', '东直门',search_strategy=sort_by_combined_cost)

"path found with 8 stations: ['北沙滩', '奥林匹克公园', '奥体中心', '北土城', '安华桥', '鼓楼大街', '安定门', '雍和宫', '东直门'] with number of transfers: 2"

####  综合最优路线：假设换乘对结果影响系数为3

In [96]:
def sort_by_combined_cost(pathes,transfer_cost=3):  
    def combined_cost(path):
       # print("path[0]",path[0])
       # print("path[1]",path[1])
        return (len(path[0])-1)+(len(path[1])-1)*transfer_cost
    return sorted(pathes,key=combined_cost)
standard_bfs(net, '北沙滩', '东直门',search_strategy=sort_by_combined_cost)

"path found with 8 stations: ['北沙滩', '奥林匹克公园', '安立路', '关庄', '望京西', '芍药居', '光熙门', '柳芳', '东直门'] with number of transfers: 1"